In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from bs4 import BeautifulSoup # Превращалка html в текст.
import re # Регулярные выражения.

import pymorphy2
import pymystem3

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (18,12)

In [85]:
train = pd.read_csv('train.csv', sep='\t', encoding='utf8', index_col='id');

In [86]:
train.head()

,name,description,target
id,,,
0,Заведующий отделом/секцией в магазин YORK (Уру...,<p><strong>В НОВЫЙ МАГАЗИН YORK (хозтовары) пр...,1
1,Наладчик станков и манипуляторов с ПУ,Обязанности:работа на токарных станках с ЧПУ T...,0
2,Разработчик С++ (Криптограф),<strong>Требования:</strong> <ul> <li>Опыт про...,0
3,Фрезеровщик,<p>Условия:</p> <ul> <li>На работу вахтовым ме...,0
4,Мерчендайзер/продавец-консультант,<p><strong>Компания Палладиум Стандарт - призн...,1


In [87]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 0 to 199999
Data columns (total 3 columns):
name           200000 non-null object
description    200000 non-null object
target         200000 non-null int64
dtypes: int64(1), object(2)
memory usage: 6.1+ MB


In [88]:
def preproc(df_init):
    df_preproc = df_init.copy()
    
    df_preproc['name'].replace(r'\([^()]*\)', '', inplace=True, regex=True) #убирает данные в скобках в столбце 'name' там часто мусор                                                                 
    df_preproc['name'].replace(r'[/]', ' ', inplace=True, regex=True) #убираем слеши
    df_preproc['name'].replace(r'\W[-]\W', '-', inplace=True, regex=True) #заменяем кострукцию ' - ' на '-'
    
    df_preproc['description'].replace(r'<li>', ' ', inplace=True, regex=True)
    
    return df_preproc

In [89]:
df_preproc = train.pipe(preproc)

In [90]:
%%time
df_preproc['description'] = df_preproc['description'].map(lambda stt: BeautifulSoup(stt, "html5lib").get_text())

Wall time: 11min 32s


In [91]:
df_preproc.head(-5)

,name,description,target
id,,,
0,Заведующий отделом секцией в магазин YORK,В НОВЫЙ МАГАЗИН YORK (хозтовары) приглашаем на...,1
1,Наладчик станков и манипуляторов с ПУ,Обязанности:работа на токарных станках с ЧПУ T...,0
2,Разработчик С++,Требования: Опыт программирования на С++ Зн...,0
3,Фрезеровщик,Условия: На работу вахтовым методом в г. Кал...,0
4,Мерчендайзер продавец-консультант,Компания Палладиум Стандарт - признанный лидер...,1
5,Мастер по эксплуатации зданий,Обязанности: Обеспечение бесперебойной работ...,0
6,Торговый представитель,"НА СТАБИЛЬНОЕ И РАСПРОСТРАНЕННОЕ НАПРАВЛЕНИЯ ""...",1
7,Торговый представитель,Обязанности: Функционал работы торгового пре...,1
8,Менеджер по продажам,"Обязанности: Продажа садовой, лесопарковой т...",1


In [25]:
""" 
Вот так делать не нужно, работает в 10 раз медленее.
%%time #20000 - Wall time: 11min 9s
for i in range (0, 20000):
    bs=BeautifulSoup(df_des['description'][i], "html5lib").get_text()
    df_preproc.loc[[i],'description'] = bs """

' \nВот так делать не нужно, работает в 10 раз медленее.\n%%time #20000 - Wall time: 11min 9s\nfor i in range (0, 20000):\n    bs=BeautifulSoup(df_des[\'description\'][i], "html5lib").get_text()\n    df_preproc.loc[[i],\'description\'] = bs '

In [93]:
test = pd.read_csv('test.csv', sep='\t', encoding='utf8', index_col='id')
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170179 entries, 200000 to 370178
Data columns (total 2 columns):
name           170179 non-null object
description    170179 non-null object
dtypes: object(2)
memory usage: 3.9+ MB


In [94]:
# обработка файла test
df_test = test.pipe(preproc)

In [95]:
df_test['description'] = df_test['description'].map(lambda stt: BeautifulSoup(stt, "html5lib").get_text())

In [96]:
def text_to_wordlist(text):
        
    text = re.sub('[^а-яёА-ЯЁ]', ' ', text)
    words = text.lower().split()

    return words

In [97]:
import nltk.data

In [98]:
def text_to_sentences(text):
        
    tokenizer = nltk.data.load('punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(text.strip())
    
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(text_to_wordlist(raw_sentence))

    return sentences

In [99]:
%%time
sentences = []

for description in df_preproc['description']:
    sentences += text_to_sentences(description)

Wall time: 3min 21s


In [100]:
%%time
for description in df_test["description"]:
    sentences += text_to_sentences(description)

Wall time: 8min 51s


In [101]:
# В конце сбора sentences будет list list'ов (список списков) - как и пример выше.
# (Повторюсь) каждый элемент списка sentences - предложение, но представленное в виде списка слов - потому список

# выведем количество элементов этого массива (оно же - количество предложений во всех текста)
print (len(sentences))

2304353


In [102]:
# а так же посмотрим на сам массив
print ('\t Первый элемент массива')
print (sentences[0])
print ()
print ('\t Второй элемент массива')
print (sentences[1])

	 Первый элемент массива
['в', 'новый', 'магазин', 'хозтовары', 'приглашаем', 'на', 'постоянную', 'работу', 'руководителя', 'секции', 'хозтовары']

	 Второй элемент массива
['обязанности', 'организация', 'эффективного', 'и', 'культурного', 'обслуживания', 'покупателей', 'организация', 'приемки', 'сдачи', 'товаров', 'на', 'склад', 'консультация', 'покупателей', 'по', 'вопросам', 'касающимся', 'оказываемых', 'услуг', 'контроль', 'своевременной', 'подачи', 'товаров', 'в', 'торговую', 'секцию', 'проверка', 'качества', 'сроков', 'годности', 'тмц', 'наличие', 'маркировок', 'ценников', 'на', 'товарах', 'организация', 'бесперебойной', 'работы', 'товарной', 'секции', 'и', 'участка', 'склада', 'закрепленного', 'за', 'ней', 'контроль', 'за', 'сохранностью', 'товаров', 'торгового', 'оборудования', 'и', 'прочих', 'материальных', 'ценностей', 'инвентаризация', 'товаров', 'требования', 'знание', 'товарной', 'группы', 'знание', 'правил', 'приемки', 'товара', 'опыт', 'работы', 'в', 'розничной', 'торгов

In [103]:
%%time
# симортируем соответствующую функцию из модуля gensim, который должен быть установлен

from gensim.models.word2vec import Word2Vec 

# список параметров, которые можно менять по вашему желанию
num_features = 300  # итоговая размерность вектора каждого слова
min_word_count = 5  # минимальная частотность слова, чтобы оно попало в модель
num_workers = 8     # количество ядер вашего процессора, чтоб запустить обучение в несколько потоков
context = 10        # размер окна 
downsampling = 1e-3 # внутренняя метрика модели

model = Word2Vec(sentences, workers=num_workers, size=num_features,
                 min_count=min_word_count, window=context, sample=downsampling)

Wall time: 1h 24min 53s


In [104]:
# Финализируем нашу модель. Ее нельзя будет доучить теперь, но она начнет занимать гораздо меньше места
model.init_sims(replace=True)

In [71]:
model.most_similar(positive=['улица', 'место'], negative=['работа'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('зависит', 0.9436490535736084),
 ('выше', 0.9310531616210938),
 ('быстрый', 0.9222092628479004),
 ('сплоченный', 0.921920895576477),
 ('стабильный', 0.9217009544372559),
 ('хороший', 0.9115757346153259),
 ('неограниченный', 0.9109642505645752),
 ('ограничений', 0.9101443886756897),
 ('потолка', 0.9079972505569458),
 ('средний', 0.9065406322479248)]

In [72]:
# давайте просмотрим вектор одного из слов
# его длину
print (len(model['улица']))

# и сам вектор
#print (model['улица'])

300


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [73]:
# Словарь - все слова которые участвуют в модели можно просмотреть так
#model.wv.index2word

In [105]:
def text_to_vec(words, model, size):
    text_vec = np.zeros((size,), dtype="float32")
    n_words = 0

    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model[word])
    
    if n_words != 0:
        text_vec /= n_words
    return text_vec

In [106]:
def texts_to_vecs(texts, model, size):
    texts_vecs = np.zeros((len(texts), size), dtype="float32")
    
    for i, text in enumerate(texts):
        texts_vecs[i] = text_to_vec(text, model, size)

    return texts_vecs

In [107]:
# действительно работает, сделаем для всех текстов из train
train_like_word_list = [sum(text_to_sentences(text), []) for text in df_preproc['description']]

In [108]:
train_vecs = texts_to_vecs(train_like_word_list, model, num_features)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [109]:
# сделаем тоже самое для test
test_like_word_list = [sum(text_to_sentences(text), []) for text in df_test['description']]

In [110]:
test_vecs = texts_to_vecs(test_like_word_list, model, num_features)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [111]:
# Воспользуемся train_vecs, test_vecs, train["sentiment"] 
#    как матрица фичей обучающей выборки, матрица фичей тестовой выборки, таргет для обучающей выборки соответственно

# Стандартный случайный лес в таком случае
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100, n_jobs=8)
forest = forest.fit(train_vecs, train["target"])
predict = forest.predict(test_vecs)

# И вот задача решена

In [112]:
predict.shape

(170179,)

In [113]:
test_out = df_test.drop(['description'], axis=1)
test_out = test_out.drop(['name'], axis=1)
test_out['target'] = predict
test_out.loc[:, 'target'].value_counts()

0    88836
1    81343
Name: target, dtype: int64

In [114]:
test_out.to_csv('res_01.csv')

In [97]:
i=17667
print(df_des['description'][i])
print()
#delscript = re.compile("<script.*?>.+?</script>", re.S)
bs=BeautifulSoup(df_des['description'][i], "html5lib").get_text()
print (bs)

<p>Требуется рабочий на мебельное производство. Необходимы знания и умения работы на присадочном станке!<br />Предлагаем возможность карьерного роста и обучение в процессе работы.</p> <p> </p> <p><strong>Обязанности:</strong></p> <ul> присадка деталей</li> </ul> <p><strong>Требования:</strong></p> <ul> образование не ниже среднего</li> аккуратность</li> исполнительность</li> умение работать в коллективе</li> </ul> <p><strong>Условия:</strong></p> <ul> график 5-2 с 9 до 18 часов</li> оформление по ТК</li> </ul> <ul> Опыт работы от 3 лет</li> </ul>

Требуется рабочий на мебельное производство. Необходимы знания и умения работы на присадочном станке!Предлагаем возможность карьерного роста и обучение в процессе работы.   Обязанности:  присадка деталей  Требования:  образование не ниже среднего аккуратность исполнительность умение работать в коллективе  Условия:  график 5-2 с 9 до 18 часов оформление по ТК   Опыт работы от 3 лет 


In [39]:
def getMeaningfullWords(text):
    words=[]
    tokens=re.findall('[А-Яа-яЁё]+\-[А-Яа-яЁё]+|[А-Яа-яЁё]+', text)
    for t in tokens:
        pv=morph.parse(t)
        for p in pv:
            if p.tag.POS in ['ADJF', 'NOUN', 'VERB']:
                words.append(p.normal_form)
                break
    return words

In [40]:
def word_vector(df_init):
    df = df_init.copy()
    df['name'].replace(df['name'], [' '.join(getMeaningfullWords(df['name']))], inplace=True, regex=True)
    return df

In [41]:
df = df_preproc.pipe(word_vector)

TypeError: expected string or bytes-like object

In [ ]:
Прошу помочь с ошибкой при подготовке данных (после очистки тегов)
def tokenized(df_input):
    vectorized = CountVectorizer(ngram_range=(1,2), tokenizer=my_tokenizer)
    #vectorized = TfidfTransformer(norm=None, smooth_idf=False)
    
    vectorized_matrix = vectorized.fit_transform(df_input["texts_without_tags"])
    vectorized_items = vectorized.vocabulary_.items()
    vectorized_matrix = vectorized_matrix.toarray()
    
    df_output = pd.DataFrame(
        vectorized_matrix,
        #np.asarray(vectorized_matrix),
        columns=[x[0] for x in sorted(vectorized_items, key=lambda x: x[1])]
    )
    
    df_output['name'] = df_input['name']
    df_output['sample'] = df_input['sample']
    df_output['target'] = df_input['target']
    
    return df_output;




import pickle
f = open(r'file.txt', 'wb')
obj = Model
pickle.dump(obj, f)
f.close() (edited)
загрузить
obj = pickle.load(f)



У меня падает ядро именно на Random Forest
живёт 2-3 минуты и всё

%%time
print (‘Обучаем Random Forest...\n’)
from sklearn.ensemble import RandomForestClassifier
# Делаем random forest на 100 деревьев
forest = RandomForestClassifier(n_estimators = 100, n_jobs = -1)
# В версии с 5 000 фич обучение заняло 22 минут 12 секунд, старт в 23-37
forest = forest.fit( train_data_features, train_df[‘target’] )
print (‘Саш, я обучилась!’)